<a href="https://colab.research.google.com/github/darknet-doll/notebook_error_logger/blob/main/Sample_Notebook_Impact_Logger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [5]:
# Import from package from error log package from github.
# Ensure error_logs.db is not in current runtime.
# !pip install -e . # For local dev only.
!pip install git+https://github.com/darknet-doll/notebook_error_logger.git --upgrade
from notebook_error_logger import start_logger
from notebook_error_logger.airtable_sync import write_airtable, read_airtable

# Import python packagess.
import sqlite3, pandas as pd

# Credentials

# Local dev only.
!pip install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()

if os.getenv("LOCAL_DEV") == "true":
    print("Local dev mode: ON")
    AIRTABLE_TOKEN = os.getenv("AIRTABLE_TOKEN")
    AIRTABLE_BASE_ID = os.getenv("AIRTABLE_BASE_ID")
else:
    # Colab
    from google.colab import userdata
    AIRTABLE_TOKEN = userdata.get("AIRTABLE_TOKEN")
    AIRTABLE_BASE_ID = userdata.get("AIRTABLE_BASE_ID")

TABLE_NAME = "Errors"

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/darknet-doll/notebook_error_logger.git to /private/var/folders/sq/3cdvc0vx48xcfjxg6c93b61w0000gn/T/pip-req-build-o51oazdb
  Running command git clone --filter=blob:none --quiet https://github.com/darknet-doll/notebook_error_logger.git /private/var/folders/sq/3cdvc0vx48xcfjxg6c93b61w0000gn/T/pip-req-build-o51oazdb
  Resolved https://github.com/darknet-doll/notebook_error_logger.git to commit f8e2f167dd927913293e5872619d59cd7f4a6956
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
Local dev mode: ON


# Log Error
This section showcases how you can set up your notebook to start logging errors locally to a SQLite database

In [6]:
# Start logging errors by creating project_name field.
logger = start_logger("Sample Notebook")

In [7]:
# Testing errors.

1 / 0 # trigger an error.

# a = er ror_syntax # undefined variable cannot be assigned.

ZeroDivisionError: division by zero

In [8]:
# Reviewing errors logged in error_logs.db dataframe for this project in SQLlite.
project_log_df = pd.read_sql_query("SELECT * FROM errors", sqlite3.connect("error_logs.db"))
project_log_df

,id,project_type,project_name,error_type,date
0,1,data science notebook,Sample Notebook,ZeroDivisionError,2025-11-21


# Airtable Connection
This section showcases how you can append your current project’s errors to your database. *I'm using Airtable as mine but you can skip this part if you decide to use a different database*.

In [12]:
# Write local project_log_df to airtable.
write_airtable(project_log_df,
               airtable_token=AIRTABLE_TOKEN,
               base_id=AIRTABLE_BASE_ID
               )


✅ All records uploaded successfully to Airtable.


In [13]:
# Read records from table.
# Validate read from link https://airtable.com/app8lLddOSQt2Fhaz/shrAvfz8PwokANwqR/tblcNUsattIK0IKun.
df_from_airtable = read_airtable(airtable_token=AIRTABLE_TOKEN,
                   base_id=AIRTABLE_BASE_ID)

# Review relevant fields.
df_from_airtable[[
    'count_id',
    'date',
    'lookup_project_type',
    'project_name',
    'error_type'
    ]].sort_values(by='count_id')

✅ Retrieved 8 records from Airtable.


,count_id,date,lookup_project_type,project_name,error_type
0,1,2025-11-12,[data science notebook],Sample Notebook,ZeroDivisionError
2,2,2025-11-13,[data science notebook],Sample Notebook,NameError
3,3,2025-11-13,[maker],Test Prototype,PrototypeError
5,4,2025-11-17,[data science notebook],Sample Notebook,ZeroDivisionError
1,5,2025-11-17,[data science notebook],Sample Notebook,ZeroDivisionError
4,7,NaN,NaN,NaN,NaN
6,8,NaN,NaN,NaN,NaN
7,9,2025-11-21,[data science notebook],Sample Notebook,ZeroDivisionError
